# Finding the Optimised Stock Portfolio among a list of stocks

Under Mean-Variance Portfolio Theorem, the optimised stock portfolio is the stock portfolio with the highest Sharpe ratio. This is a computer programme to find the optimised stock portfolio, formed by stocks in a specified stock list (which is decided by the user). It will compute the weight of each stock in the optimised portfolio. This programme consist of 2 parts. The 1st part is using traditional matrix methods to solve for those weights. This method has the limitation of assuming that there are no transaction costs and assuming that lending cost and borrowing cost is the same. The 2nd part accounts for other factors in reality, such as transaction costs, dividend tax and the fact that lending cost and borrowing cost is not the same. This part exports the necessary data to Microsoft Excel. In Excel, you can write your own formulas (which accounts for transaction costs, dividend tax, a speficied borrowing rate or even other factors) to compute the mean and standard deviation of the portfolio's return, and the portfolio's Sharpe ratio. Lastly, you can use the "Solver" function in Excel to solve for the weight of each stock in the optimised portfolio.

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime

In [5]:
import yfinance as yf

First we have to decide a stock list that our portfolio choose stocks from, such as the the constituents of Dow Jones Industrial Average (DJIA), or the constituents of S&P 500 index, as shown in the code cell below. Then we get the monthly adjusted closing price (over the past 6 years) of those stocks. For example, in the code cell below, the S&P 500 constituents at the end of 2022 is chosen as the stock list and then we get the monthly adjusted closing price of this stock list from 2016-12-01 to 2022-12-31.

In [64]:
#DJIA stock list
djia_lst = ['MMM', 'AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PG', 'CRM', 'TRV', 'UNH', 'VZ', 'V', 'WBA', 'WMT', 'DIS']

#The following is the code to get the list of constituents of S&P 500 index as of the end of 2022.
#You can change the year in the following "for block" to get the constituents list as of the end of your desired year. 
fhand = open('.\Documents\Stock\Backtests\SP_List.csv', 'r')
sp = []
for line in fhand:
    line = line.rstrip()
    line = line.split(',')
    if line[0] == '2021':
        splst = []
        for i in line:
            if i != '' and i != '2021':
                splst.append(i)
        print(splst)

#Get the monthly adjusted closing price of the chosen stock list. In this example, the S&P 500 stock list is chosen.
#E.g. below is getting the data with start date 2016-12-01 and end date 2022-12-31. Change the year and date if needed.
#You can change "splst" below to "djia_lst" if you want to choose the DJIA stock list instead of the S&P 500 one.
djia_s = yf.download(splst, 
                      start='2016-12-01', 
                      end='2022-12-31', 
                      progress=False,
                      interval="1mo",
).drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)

fhand.close()


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'TWTR', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'FRC', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK-B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'SBNY', 'CHRW', 'CDNS', 'CZR', 'PBCT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'GPS', 'COO', 'CPRT', 'GLW', 'CTVA', 'PVH', 'COST', 'CTRA', 'CCI', 'CSX', '

In the following 2 code cells, we are going to compute the dividend yield of the last year for the stocks, as an estimate of the dividend yield of the coming year. This is used to account for the effect of dividend tax on the return.

In [15]:
#Get the annual total dividend of the last year
# Define the start and end dates (full year of 2022)
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)

# Download the dividends for each symbol and concatenate the results
dfs = []
for symbol in sorted(djia_lst):
    stock = yf.Ticker(symbol)
    dividends = stock.history(start=start_date, end=end_date)["Dividends"].to_frame(name=symbol)
    dfs.append(dividends)

#drop columns with NaN and/or zero values 
df = pd.concat(dfs, axis=1).dropna(axis=1, how='all')
#df = df.loc[:, (df != 0).any(axis=0)]
df.index.name = "Date"

#add 'Annual Dividend' as the last row 

df.loc['Annual Dividend'] = df.sum(axis=0)

#create a new frame with with 'Annual Dividend' row only

annual_div = df.loc['Annual Dividend'].to_frame()
div = annual_div.transpose()
print(div)
print(div.shape)

                 AAPL  AMGN   AXP   BA   CAT  CRM  CSCO   CVX  DIS  DOW  ...  \
Annual Dividend  0.91  7.76  1.99  0.0  4.62  0.0  1.51  5.68  0.0  2.8  ...   

                 MRK  MSFT    NKE     PG   TRV  UNH      V     VZ    WBA   WMT  
Annual Dividend  2.8  2.54  1.255  3.609  3.67  6.4  1.575  2.573  1.916  2.24  

[1 rows x 30 columns]
(1, 30)


In [18]:
#Compute the dividend yield of the last year
djia_close = yf.download(djia_lst, 
                      start='2021-12-01', 
                      end='2021-12-31', 
                      progress=False,
                      interval="1mo",
).drop(columns = ['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
print(djia_close)
dyield = div.to_numpy()/djia_close.to_numpy()
dyield = pd.DataFrame(div.to_numpy()/djia_close.to_numpy(), columns=div.columns)
print(dyield)
    

                 Close                                                  \
                  AAPL        AMGN         AXP          BA         CAT   
Date                                                                     
2021-12-01  177.570007  224.970001  163.600006  201.320007  206.740005   

                                                                      ...  \
                   CRM       CSCO         CVX         DIS        DOW  ...   
Date                                                                  ...   
2021-12-01  254.130005  63.369999  117.349998  154.889999  56.720001  ...   

                                                                       \
                  MRK        MSFT         NKE          PG         TRV   
Date                                                                    
2021-12-01  76.639999  336.320007  166.669998  163.580002  156.429993   

                                                                  
                   UNH           V

In the following code cell, we compute the continuously compounding return (difference in logarithms) of the DJIA stocks, find the mean and sample standard deviation (SD) of the returns (over the past 5 years) of the stocks and compute the sample variance-covariance matrix of the returns of the stocks. Then we proceed to use matrix methods to solve for the weight of each stock in the optimised portfolio such that the Sharpe ratio is maximised.

In [63]:
#Compute the continuously compounding return (difference in logarithms) of the stocks
lst = np.log(djia_s)
ri = lst.diff() 
ri.index = pd.to_datetime(ri.index)

#Get the mean & sample standard deviation (SD) of the returns, over the past 5 years, of the stocks.
#Please change the "years" below accordingly, if needed.
rihist = ri['Adj Close'].loc['2017-01-01':'2021-12-31'].dropna(axis='columns')
mean = rihist.mean(axis=0)*12
sd = rihist.std(axis=0)*np.sqrt(12)
print(mean)

#Compute the sample variance-covariance matrix (sigma) of the returns of the stocks
cov = rihist.cov()

#Compute the inverse of sigma
cov_inv = pd.DataFrame(np.linalg.pinv(cov.values), cov.columns, cov.index)

#Get the risk-free interest rate at year-end
#Please change the "year" below accordingly, if needed.
rf = np.log(1+yf.download("^TNX", 
                      start='2021-12-01', 
                      end='2021-12-31', 
                      progress=False,
                      interval="1mo",
).drop(columns = ['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)/100)
rfnp = rf.to_numpy()
print(float(rfnp))

miumrf = mean.sub(float(rfnp))
print(miumrf)

z = cov_inv.dot(miumrf)

w = z.div(z.sum())
print(w)

#If short-selling is allowed, you do not need to use the following "while" block. You can "comment" this block. 

while (w<0).any():
    w = w[w>0]
    d = rihist[w.index]
    c = d.cov()
    c_inv = pd.DataFrame(np.linalg.pinv(c.values), c.columns, c.index)
    m = mean[w.index].sub(float(rfnp))
    z = c_inv.dot(m)
    w = z.div(z.sum())

#This print statement shows the weight of each stock in the optimised portfolio when short-selling is not allowed.
print(w)
print(w.shape)
stocklst = list(w.index)
print(stocklst)

#The following codes print the continuous compounding return of the coming year after portfolio formation.
#Please change the "years" below accordingly, if needed.
print(np.exp(ri['Adj Close'].loc['2022-01-01':'2022-12-31'][stocklst].sum(axis=0)))
result = w.dot(np.exp(ri['Adj Close'].loc['2022-01-01':'2022-12-31'][stocklst].sum(axis=0)))
print(np.log(result))


A       0.259140
AAL    -0.184227
AAP     0.074843
AAPL    0.374827
ABBV    0.200189
          ...   
YUM     0.174374
ZBH     0.049311
ZBRA    0.387469
ZION    0.100613
ZTS     0.309548
Length: 482, dtype: float64
0.015006831758918347
A       0.244134
AAL    -0.199234
AAP     0.059836
AAPL    0.359820
ABBV    0.185182
          ...   
YUM     0.159367
ZBH     0.034304
ZBRA    0.372462
ZION    0.085606
ZTS     0.294541
Length: 482, dtype: float64
A       0.006032
AAL    -0.005965
AAP    -0.001877
AAPL    0.002994
ABBV   -0.002215
          ...   
YUM     0.005866
ZBH    -0.020285
ZBRA    0.012596
ZION   -0.002457
ZTS     0.014020
Length: 482, dtype: float64
ABT     0.131955
AWK     0.025369
COST    0.201556
ETSY    0.023804
FTNT    0.159434
LRCX    0.053491
NEE     0.184322
SEDG    0.104809
TMO     0.099338
TSLA    0.015922
dtype: float64
(10,)
['ABT', 'AWK', 'COST', 'ETSY', 'FTNT', 'LRCX', 'NEE', 'SEDG', 'TMO', 'TSLA']
ABT     0.793181
AWK     0.821079
COST    0.809502
ETSY    0.54709

This is the start of the 2nd part of this programme, which accounts for other factors in reality, such as transaction costs and the fact that lending cost and borrowing cost is not the same. 

In [243]:
#This part exports the necessary data to Microsoft Excel.
#In Excel, you can write your own formulas (which accounts for transaction costs, dividend tax, a speficied borrowing rate or even other factors) 
#to compute the mean and standard deviation of the portfolio's return, and the portfolio's Sharpe ratio.
with pd.ExcelWriter('DJIA_2023.xlsx') as writer:  
    djia_s.to_excel(writer, sheet_name='s')
    rihist.to_excel(writer, sheet_name='r')
    cov.to_excel(writer, sheet_name='cov')
    mean.to_excel(writer, sheet_name='miu')
    sd.to_excel(writer, sheet_name='sd')
    dyield.to_excel(writer, sheet_name='div')

Then proceed to Microsoft Excel to do the remaining jobs: 
1. compute the portfolio's monthly variance (and hence SD) by setting the weight of each stock, and by using the above covariance matrix. 
2. Then compute the annual SD by multiplying the monthly SD by square root of 12. 
3. Then write formulas (which accounts for transaction costs and dividend tax, a speficied borrowing rate or other factors) to compute the portfolio mean return.
4. After that, write forumlas (by using a specified borrowing rate and/or lending rate) to compute the Sharpe ratio of the portfolio
5. Lastly, use "Solver" to solve for the weight of each stock such that the Sharpe ratio is maximised.

In [339]:
companies=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
table  = companies[0]
splst = table
print(list(splst['Symbol']))

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BG', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'C